<a href="https://colab.research.google.com/github/akatari2024-debug/Akhilesh-katari/blob/main/Copy_of_assignment2_camera_pose_planar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2 — Camera Pose from a Planar ObjectColab-ready notebook with a Gradio UI to click points, estimate pose via Homography→Pose and OpenCV `solvePnP`, and visualize overlays.Run all cells in order. The first cell installs compatible packages for Colab.

In [1]:
import cv2
import json
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt

# This helper function is correct as is.
def homography_to_pose(H, K):
    """Decomposes a homography matrix to a rotation matrix and a translation vector."""
    H_norm = np.linalg.inv(K) @ H
    h1, h2, h3 = H_norm[:, 0], H_norm[:, 1], H_norm[:, 2]
    lam = 1.0 / np.linalg.norm(h1)
    r1 = lam * h1
    r2 = lam * h2
    r3 = np.cross(r1, r2)
    t = lam * h3
    R = np.column_stack((r1, r2, r3))
    U, _, Vt = np.linalg.svd(R)
    R = U @ Vt
    return R, t

# This helper function is correct as is.
def draw_points_on_image(image, points):
    """Draws numbered circles for each point on a copy of the image."""
    if image is None:
        return None

    overlay = image.copy()
    for i, (x, y) in enumerate(points):
        cv2.circle(overlay, (int(x), int(y)), 8, (0, 255, 0), -1)
        cv2.putText(overlay, str(i), (int(x) + 10, int(y) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    return overlay

# 🔴 NEW HELPER FUNCTION TO FIX BLACK IMAGE ISSUE
def fix_image_type(image):
    """
    Robustly converts a NumPy image array to uint8 for display.
    Handles float arrays in both 0-1 and 0-255 ranges.
    """
    if image is None:
        return None
    # If it's already uint8, do nothing
    if image.dtype == np.uint8:
        return image

    # If it's a float, we need to scale it correctly
    if image.dtype == np.float32 or image.dtype == np.float64:
        # Check if the max value is around 1.0, indicating a normalized image
        if image.max() <= 1.0:
            image = (image * 255).astype(np.uint8)
        else: # Otherwise, assume it's already in the 0-255 range
            image = image.astype(np.uint8)
        return image

    # For other types, try a direct conversion
    return image.astype(np.uint8)


def process(image, intrinsics_json, points):
    """
    Estimates camera pose from an undistorted image and corresponding 2D points.
    """
    if image is None:
        return "Please upload an image first!", None, None
    if len(points) < 4:
        return f"Need at least 4 points! Currently have {len(points)}.", None, None

    try:
        intr = json.loads(intrinsics_json)
        original_K = np.array(intr["K"])
        original_dist = np.array(intr["dist"])

        h, w = image.shape[:2]
        K, _ = cv2.getOptimalNewCameraMatrix(original_K, original_dist, (w, h), 1, (w, h))
        dist = np.zeros(5)

        img_pts = np.array(points, dtype=np.float32)

        model_pts = np.array([
            [0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0]
        ], dtype=np.float32)

        success, rvec, tvec = cv2.solvePnP(model_pts, img_pts, K, dist)
        if not success:
            return "solvePnP failed. Check if points are co-linear.", None, None

        R_cv, _ = cv2.Rodrigues(rvec)
        overlay = image.copy()
        proj_pts, _ = cv2.projectPoints(model_pts, rvec, tvec, K, dist)

        for i, pt in enumerate(proj_pts):
            x, y = int(pt[0][0]), int(pt[0][1])
            cv2.circle(overlay, (x, y), 8, (0, 0, 255), -1)
            cv2.putText(overlay, f"P{i}", (x + 10, y + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        for i, (x, y) in enumerate(img_pts.astype(int)):
            cv2.circle(overlay, (x, y), 8, (0, 255, 0), -1)
            cv2.putText(overlay, f"C{i}", (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        axis_3d = np.float32([[0,0,0], [1,0,0], [0,1,0], [0,0,-1]]).reshape(-1, 3) * 0.75
        axis_2d, _ = cv2.projectPoints(axis_3d, rvec, tvec, K, dist)
        origin = tuple(axis_2d[0].ravel().astype(int))
        cv2.line(overlay, origin, tuple(axis_2d[1].ravel().astype(int)), (0, 0, 255), 3)
        cv2.line(overlay, origin, tuple(axis_2d[2].ravel().astype(int)), (0, 255, 0), 3)
        cv2.line(overlay, origin, tuple(axis_2d[3].ravel().astype(int)), (255, 0, 0), 3)

        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111, projection="3d")

        R_cam_to_world = R_cv.T
        t_cam_in_world = -R_cam_to_world @ tvec

        ax.quiver(0, 0, 0, 1, 0, 0, color="k", label="World X", arrow_length_ratio=0.1)
        ax.quiver(0, 0, 0, 0, 1, 0, color="k", label="World Y", arrow_length_ratio=0.1)
        ax.quiver(0, 0, 0, 0, 0, 1, color="k", label="World Z", arrow_length_ratio=0.1)

        cam_x, cam_y, cam_z = t_cam_in_world.flatten()
        ax.scatter(cam_x, cam_y, cam_z, c='purple', marker='o', s=100, label='Camera Center')
        cam_axes = R_cam_to_world * 0.5
        ax.quiver(cam_x, cam_y, cam_z, cam_axes[0, 0], cam_axes[1, 0], cam_axes[2, 0], color="r", label="Cam X")
        ax.quiver(cam_x, cam_y, cam_z, cam_axes[0, 1], cam_axes[1, 1], cam_axes[2, 1], color="g", label="Cam Y")
        ax.quiver(cam_x, cam_y, cam_z, cam_axes[0, 2], cam_axes[1, 2], cam_axes[2, 2], color="b", label="Cam Z")

        ax.set_title("Camera Pose in World Coordinates")
        ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
        ax.legend()
        ax.set_aspect('equal', adjustable='box')

        reprojection_error = np.mean(np.linalg.norm(img_pts - proj_pts.squeeze(), axis=1))
        out_text = (f"✅ Pose Estimated!\n\n"
                    f"Reprojection Error: {reprojection_error:.2f} pixels\n\n"
                    f"Camera Position (in world coords):\n"
                    f"  t = [{t_cam_in_world[0][0]:.3f}, {t_cam_in_world[1][0]:.3f}, {t_cam_in_world[2][0]:.3f}]\n\n"
                    f"Camera Rotation (world-to-camera):\n{np.round(R_cv, 3)}")

        # 🔴 Applying the robust fix before returning the image
        return out_text, fix_image_type(overlay), fig

    except Exception as e:
        return f"An error occurred: {str(e)}", None, None

default_intrinsics = """{
  "K": [
    [1000.0, 0.0, 500.0],
    [0.0, 1000.0, 500.0],
    [0.0, 0.0, 1.0]
  ],
  "dist": [0.0, 0.0, 0.0, 0.0, 0.0]
}"""
with gr.Blocks(title="Pose Estimation Tool", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📷 Camera Pose Estimation Tool")
    gr.Markdown("Upload an image containing a flat, square object. Then **click on the 4 corners of the square in order** (e.g., bottom-left, bottom-right, top-right, top-left).")

    original_image_state = gr.State(None)
    undistorted_image_state = gr.State(None)
    point_state = gr.State([])

    with gr.Row():
        with gr.Column(scale=2):
            img_display = gr.Image(type="numpy", label="Upload Image & Click 4 Corners")
            point_info = gr.Textbox(label="Selected Points", interactive=False)
        with gr.Column(scale=1):
            intr = gr.Textbox(label="Camera Intrinsics (JSON)", lines=12, value=default_intrinsics)

    with gr.Row():
        clear_btn = gr.Button("Clear Points")
        estimate_btn = gr.Button("Estimate Pose", variant="primary")

    with gr.Row():
        with gr.Column(scale=1):
            out_text = gr.Textbox(label="Pose Estimation Results", lines=10)
        with gr.Column(scale=2):
            out_img = gr.Image(label="Image with Pose Overlay")
            out_fig = gr.Plot(label="3D Visualization of Camera Pose")

    def on_image_upload(image, intrinsics_str):
        if image is None:
            gr.Warning("Image upload failed. Please try again.")
            return None, None, [], "Upload an image."
        try:
            intr = json.loads(intrinsics_str)
            K = np.array(intr["K"])
            dist = np.array(intr["dist"])
            h, w = image.shape[:2]
            new_K, _ = cv2.getOptimalNewCameraMatrix(K, dist, (w, h), 1, (w, h))
            undistorted = cv2.undistort(image, K, dist, None, new_K)

            # 🔴 Applying the robust fix after undistorting
            undistorted_fixed = fix_image_type(undistorted)

            return image, undistorted_fixed, [], "Click 4 points on the image."
        except Exception as e:
            gr.Warning(f"Failed to undistort image: {e}")
            return image, image, [], "Error: Could not process intrinsics."

    def collect_points(evt: gr.SelectData, points, image):
        if image is None:
            gr.Warning("Please upload an image first!")
            return points, "Upload an image first.", None

        new_points = points + [evt.index]

        info_text = f"Selected {len(new_points)} points.\n"
        if len(new_points) < 4:
            info_text += f"Need {4 - len(new_points)} more."
        else:
            info_text += "Ready to estimate pose!"

        overlay = draw_points_on_image(image, new_points)
        return new_points, info_text, fix_image_type(overlay)

    def clear_all_points(image):
        return [], "Points cleared.", image

    img_display.upload(
        on_image_upload,
        inputs=[img_display, intr],
        outputs=[original_image_state, undistorted_image_state, point_state, point_info]
    ).then(
        lambda x: x, inputs=undistorted_image_state, outputs=img_display
    )

    img_display.select(
        collect_points,
        inputs=[point_state, undistorted_image_state],
        outputs=[point_state, point_info, img_display]
    )

    clear_btn.click(
        clear_all_points,
        inputs=[undistorted_image_state],
        outputs=[point_state, point_info, img_display]
    )

    estimate_btn.click(
        process,
        inputs=[undistorted_image_state, intr, point_state],
        outputs=[out_text, out_img, out_fig]
    )

if __name__ == "__main__":
    demo.launch(debug=False)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cb56bc26be5e498b3f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
